<a href="https://colab.research.google.com/github/vachkim/LECTURE/blob/master/MACHINE_LEARNING/Lec_BOSTON_MACHINELEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1) 데이터 불러오기

In [ ]:
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

%cd drive/'My Drive'/'Data Analysis'/'Data'
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Data Analysis/Data
data_description.txt  sample_submission.csv  test.csv  train.csv


In [ ]:
import pandas as pd
sample_submission = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
import pandas as pd
from pandas.io import gbq

# Connect to Google Cloud API and Upload DataFrame
sample_submission.to_gbq(destination_table='new_house.sample_submission', 
                  project_id='leafy-loader-286903', 
                  if_exists='replace')

1it [00:02,  2.63s/it]


In [ ]:
colnames_dict = {"1stFlrSF": "my1stFlrSF", "2ndFlrSF": "my2ndFlrSF", "3SsnPorch": "my3SsnPorch"}

In [ ]:
train = train.rename(columns=colnames_dict)
train.to_gbq(destination_table='new_house.train', 
                  project_id='leafy-loader-286903', 
                  if_exists='replace')

1it [00:02,  2.96s/it]


In [ ]:
test = pd.read_csv('test.csv')
test = test.rename(columns=colnames_dict)
test.to_gbq(destination_table='new_house.test', 
            project_id='leafy-loader-286903', 
            if_exists='replace')

1it [00:02,  2.83s/it]


In [ ]:
from google.colab import auth
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'leafy-loader-286903'
client = bigquery.Client(project=project_id)

df_train = client.query('''
  SELECT 
      * 
  FROM `leafy-loader-286903.new_house.train`
  ''').to_dataframe()

df_train.shape

(1460, 81)

In [ ]:
df_test = client.query('''
  SELECT 
      * 
  FROM `leafy-loader-286903.new_house.test`
  ''').to_dataframe()

df_test.shape

(1459, 80)

### 2) 주요 패키지 불러오기

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 3) 결측 데이터 확인

In [ ]:
def check_fill_na(data):
  new_df = data.copy()
  new_df_na = (new_df.isnull().sum() / len(new_df)) * 100
  new_df_na.sort_values(ascending=False).reset_index(drop=True)
  new_df_na = new_df_na.drop(new_df_na[new_df_na == 0].index).sort_values(ascending=False)
  return new_df_na

check_fill_na(df_train)

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageYrBlt      5.547945
GarageType       5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
BsmtQual         2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
dtype: float64

### 4) 주요 함수 정의

In [ ]:
def fill_missing(df, cols, val):
    """ val 입력값을 넣는다. """
    for col in cols:
        df[col] = df[col].fillna(val)

def fill_missing_with_mode(df, cols):
    """ 최대 빈도수를 넣는다. """
    for col in cols:
        df[col] = df[col].fillna(df[col].mode()[0])
        
def addlogs(res, cols):
    """ 로그 변환 """
    m = res.shape[1]
    for c in cols:
        res = res.assign(newcol=pd.Series(np.log(1.01+res[c])).values)   
        res.columns.values[m] = c + '_log'
        m += 1
    return res

### 5) 데이터 처리

In [ ]:
# 전체 면적 데이터 추가
df_train['TotalSF'] = df_train['TotalBsmtSF'] + df_train['my1stFlrSF'] + df_train['my2ndFlrSF']

In [ ]:
#로그 변환
loglist = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
            'TotalBsmtSF','my1stFlrSF','my2ndFlrSF','LowQualFinSF','GrLivArea',
            'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
            'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF',
            'EnclosedPorch','my3SsnPorch','ScreenPorch','PoolArea','MiscVal','YearRemodAdd','TotalSF']

df_train = addlogs(df_train, loglist)

In [ ]:
#종속변수 로그변환
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])

In [ ]:
df_train['SalePrice']

0       11.931642
1       11.283525
2       11.798112
3       12.468441
4       11.512935
          ...    
1455    11.849405
1456    12.031725
1457    12.341263
1458    11.786770
1459    11.767575
Name: SalePrice, Length: 1460, dtype: float64

In [ ]:
#결측치 처리
# 우선, 결측치가 있는 것 중, 범주형 데이터는 "None"으로 확인
fill_missing(df_train, ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", 
                        "GarageType", "GarageFinish", "GarageQual", "GarageCond",
                       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       "MasVnrType", "MSSubClass"], "None") 

# 수치형 데이터는 0으로 보간
fill_missing(df_train, ["GarageYrBlt", "GarageArea", "GarageCars",
                       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                       "MasVnrArea"], 0)
# 그 중, 일부는 빈도수로 채워 넣는다.  
fill_missing_with_mode(df_train, ["MSZoning", "KitchenQual", "Exterior1st", "Exterior2nd", "SaleType"])
fill_missing(df_train, ["Functional"],"Typ")

In [ ]:
#변수 삭제
df_train.drop(['Utilities'], axis=1, inplace=True)

In [ ]:
df_train['SalePrice']

0       11.931642
1       11.283525
2       11.798112
3       12.468441
4       11.512935
          ...    
1455    11.849405
1456    12.031725
1457    12.341263
1458    11.786770
1459    11.767575
Name: SalePrice, Length: 1460, dtype: float64

In [ ]:
#이상치 제거
df_train.drop(df_train[(df_train['OverallQual']<5) & (df_train['SalePrice']>200000)].index, inplace=True)
df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index, inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [ ]:
#재범주화
df_train['MSSubClass'] = df_train['MSSubClass'].apply(str)
df_train['YrSold'] = df_train['YrSold'].astype(str)
df_train['MoSold'] = df_train['MoSold'].astype(str)

In [ ]:
#범주형 데이터 다루기
#원핫인코딩 _ 원핫인코딩은 범주형 데이터가 너무 크지 않을 경우 사용한다
#범주형 데이터가 너무 많을 경우에는 범주끼리 묶어서 재범주화 하는 과정이 필요하다
def fix_missing_cols(in_train, in_test):
    missing_cols = set(in_train.columns) - set(in_test.columns)
    # 테스트 데이터와 훈련 데이터의 컬럼을 동일하게 하는 코드는 작성한다. 
    for c in missing_cols:
        in_test[c] = 0
    # 순서를 동일하게 만든다. 
    in_test = in_test[in_train.columns]
    return in_test

def dummy_encode(in_df_train, in_df_test):
    df_train = in_df_train
    df_test = in_df_test
    categorical_feats = [
        f for f in df_train.columns if df_train[f].dtype == 'object'
    ]
    print(categorical_feats)
    for f_ in categorical_feats:
        prefix = f_
        df_train = pd.concat([df_train, pd.get_dummies(df_train[f_], prefix=prefix)], axis=1).drop(f_, axis=1)
        df_test = pd.concat([df_test, pd.get_dummies(df_test[f_], prefix=prefix)], axis=1).drop(f_, axis=1)
        df_test = fix_missing_cols(df_train, df_test)
    return df_train, df_test

In [ ]:
df_train['SalePrice']

0       11.931642
1       11.283525
2       11.798112
3       12.468441
4       11.512935
          ...    
1451    11.849405
1452    12.031725
1453    12.341263
1454    11.786770
1455    11.767575
Name: SalePrice, Length: 1456, dtype: float64

In [ ]:
df_train, df_test = dummy_encode(df_train, df_test)
print("Shape train: %s, test: %s" % (df_train.shape, df_test.shape))

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']
Shape train: (1456, 361), test: (1459, 361)


### 6) 머신러닝 모형 개발

In [ ]:
#bagging/busting
#lightGBM/XGBoost

In [ ]:
# 종속변수 처리
y = df_train["SalePrice"]
y.sample(3)

790     12.055256
680     12.000898
1322    12.027191
Name: SalePrice, dtype: float64

In [ ]:
df_train.drop(["SalePrice"], axis=1, inplace=True)
df_test.drop(["SalePrice"], axis=1, inplace=True)

print("Shape train: %s, test: %s" % (df_train.shape, df_test.shape))

Shape train: (1456, 360), test: (1459, 360)


In [ ]:
#데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split( df_train, y, test_size=0.2, random_state=42)

In [ ]:
#LightGBM 파라미터 정의
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

In [ ]:
#모델 정의
gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
#모델 학습
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=1000)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 1	valid_0's l1: 0.333038	valid_0's l2: 0.186477
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's auc: 1	valid_0's l1: 0.331606	valid_0's l2: 0.185073
[3]	valid_0's auc: 1	valid_0's l1: 0.330192	valid_0's l2: 0.183684
[4]	valid_0's auc: 1	valid_0's l1: 0.328797	valid_0's l2: 0.18231
[5]	valid_0's auc: 1	valid_0's l1: 0.327404	valid_0's l2: 0.18095
[6]	valid_0's auc: 1	valid_0's l1: 0.326023	valid_0's l2: 0.179603
[7]	valid_0's auc: 1	valid_0's l1: 0.324652	valid_0's l2: 0.17828
[8]	valid_0's auc: 1	valid_0's l1: 0.323279	valid_0's l2: 0.176947
[9]	valid_0's auc: 1	valid_0's l1: 0.321952	valid_0's l2: 0.175652
[10]	valid_0's auc: 1	valid_0's l1: 0.320633	valid_0's l2: 0.174352
[11]	valid_0's auc: 1	valid_0's l1: 0.319239	valid_0's l2: 0.172985
[12]	valid_0's auc: 1	valid_0's l1: 0.317859	valid_0's l2: 0.171633
[13]	valid_0's auc: 1	valid_0's l1: 0.316502	valid_0's l2: 0.170309
[14]	valid_0's auc: 1	valid_0's l1: 0.315116	valid_0's l2: 0.168

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.005, max_bin=512,
              max_depth=8, metric=['l2', 'auc'], min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_iterations=100000, num_leaves=128,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=0)

In [ ]:
#모델 평가
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))

The rmse of prediction is: 0.02951


In [ ]:
#결과 제출
test_pred = np.expm1(gbm.predict(df_test, num_iteration=gbm.best_iteration_))
df_test["SalePrice"] = test_pred
df_test.to_csv("results.csv", columns=["Id", "SalePrice"], index=False)